In [3]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
from datetime import datetime
import glob
from matplotlib import pyplot as plt

In [4]:
duration1_file = "../../data_xaicha/duration1.txt"
duration2_file = "../../data_xaicha/duration2.txt"

sensor_root_dir = "../../data_xaicha/時刻変換済みセンサデータ"

## First time

In [12]:
df = pd.read_csv(duration1_file, sep="\t", header=None)
duration_df = df.rename({
    2: "start",
    3: "end",
    4: "duration",
    5: "action"
}, axis=1).drop([0, 1], axis=1)
duration_df

,start,end,duration,action
0,00:00:11.080,00:00:11.540,00:00:00.460,振り上げ
1,00:04:15.090,00:05:06.660,00:00:51.570,階段降り
2,00:05:29.900,00:06:31.190,00:01:01.290,階段上り
3,00:06:47.110,00:07:53.050,00:01:05.940,立っている
4,00:09:25.780,00:10:25.580,00:00:59.800,走っている
5,00:10:48.110,00:11:49.430,00:01:01.320,歩いている
6,00:12:31.180,00:13:39.910,00:01:08.730,座っている


In [13]:
def to_dt(string):
    return datetime.strptime(string, "%H:%M:%S.%f")

# Test

In [14]:
sensor_a_path = os.path.join(sensor_root_dir, "Conv-sensorA_1-1.csv")

sensor_a_df = pd.read_csv(sensor_a_path, header=None)

sensor_a_df = sensor_a_df.rename({
    1: "ts",
    2: "x",
    3: "y",
    4: "z",
    5: "gx",
    6: "gy",
    7: "gz"
}, axis=1).drop(0, axis=1)
sensor_a_df

/var/folders/m8/y6jc4w1s4cbb3830x0n6zygr0000gn/T/ipykernel_91656/373123072.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sensor_a_df = pd.read_csv(sensor_a_path, header=None)


,ts,x,y,z,gx,gy,gz
0,17:18:30.350,-2920,9214,-2935,205,567,-641
1,17:18:30.360,-2920,9180,-2960,174,609,-665
2,17:18:30.370,-2813,9161,-2979,131,585,-714
3,17:18:30.380,-2862,9161,-3097,150,597,-769
4,17:18:30.390,-2837,9112,-3087,144,652,-757
...,...,...,...,...,...,...,...
143498,17:42:25.330,-3199,733,9422,205,-66,5
143499,17:42:25.340,-3194,713,9368,131,-5,-2
143500,17:42:25.350,-3130,709,9339,22,-48,-14
143501,17:42:25.360,-3145,738,9383,-57,-42,-8


In [15]:
sensor_a_df["dt"] = sensor_a_df["ts"].apply(to_dt)
sensor_a_df

,ts,x,y,z,gx,gy,gz,dt
0,17:18:30.350,-2920,9214,-2935,205,567,-641,1900-01-01 17:18:30.350
1,17:18:30.360,-2920,9180,-2960,174,609,-665,1900-01-01 17:18:30.360
2,17:18:30.370,-2813,9161,-2979,131,585,-714,1900-01-01 17:18:30.370
3,17:18:30.380,-2862,9161,-3097,150,597,-769,1900-01-01 17:18:30.380
4,17:18:30.390,-2837,9112,-3087,144,652,-757,1900-01-01 17:18:30.390
...,...,...,...,...,...,...,...,...
143498,17:42:25.330,-3199,733,9422,205,-66,5,1900-01-01 17:42:25.330
143499,17:42:25.340,-3194,713,9368,131,-5,-2,1900-01-01 17:42:25.340
143500,17:42:25.350,-3130,709,9339,22,-48,-14,1900-01-01 17:42:25.350
143501,17:42:25.360,-3145,738,9383,-57,-42,-8,1900-01-01 17:42:25.360


In [16]:
# base on excel we find that 振り上げ -> 18:09:58.850 in ts
init_camera = to_dt(duration_df[duration_df["action"] == "振り上げ"].end.values[0])
init_camera

datetime.datetime(1900, 1, 1, 0, 0, 11, 540000)

In [22]:
init_sensor = to_dt("17:18:54.830")
init_sensor

datetime.datetime(1900, 1, 1, 17, 18, 54, 830000)

In [23]:
time_delta = init_sensor - init_camera
time_delta

datetime.timedelta(seconds=62323, microseconds=290000)

In [24]:
sensor_a_df["label"] = None

for duration_row in duration_df.iterrows():
    if duration_row[1].action == "振り上げ":
        continue
        
    camera_start = to_dt(duration_row[1].start)
    camera_end = to_dt(duration_row[1].end)
    
    sensor_start = camera_start + time_delta
    sensor_end = camera_end + time_delta
    
    sensor_a_df.loc[(sensor_a_df["dt"] > sensor_start) & (sensor_a_df["dt"] < sensor_end), "label"] = duration_row[1].action

In [25]:
sensor_a_df["label"].value_counts()

座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64

In [26]:
sensor_a_df[sensor_a_df["label"].notnull()].drop("dt", axis=1)

,ts,x,y,z,gx,gy,gz,label
26804,17:22:58.390,-9409,2901,1448,1113,-268,-153,階段降り
26805,17:22:58.400,-9453,2930,1512,1046,-237,-177,階段降り
26806,17:22:58.410,-9492,2984,1473,955,-213,-177,階段降り
26807,17:22:58.420,-9434,2945,1483,827,-219,-183,階段降り
26808,17:22:58.430,-9400,2940,1502,741,-182,-190,階段降り
...,...,...,...,...,...,...,...,...
83280,17:32:23.150,-3853,5445,7440,70,-91,-122,座っている
83281,17:32:23.160,-3804,5420,7405,101,-54,-86,座っている
83282,17:32:23.170,-3828,5376,7440,150,-85,-116,座っている
83283,17:32:23.180,-3833,5391,7435,180,-79,-116,座っている



# 本番

In [34]:
sensor_path_list = list(map(lambda x: os.path.join(sensor_root_dir, f"Conv-sensor{x}_1-1.csv"), "ABCDE"))
sensor_path_list

['../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorA_1-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorB_1-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorC_1-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorD_1-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorE_1-1.csv']

In [35]:
init_sensor_list = [
    "17:18:54.830", #A
    "17:18:55.210", #B
    "17:18:55.400", #C
    "17:18:55.120", #D
    "17:18:55.690", #E
]

In [36]:
save_dir = os.path.join(sensor_root_dir, "..", "tran_labeled")
os.makedirs(save_dir, exist_ok = True)

sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}_1-1.csv"), "ABCDE"))

In [37]:
def read_one_sensor_file(sensor_path, init_sensor_time_str):
    sensor_df = pd.read_csv(sensor_path, header=None)

    sensor_df = sensor_df.rename({
        1: "ts",
        2: "ax",
        3: "ay",
        4: "az",
        5: "gx",
        6: "gy",
        7: "gz"
    }, axis=1).drop(0, axis=1)
    
    sensor_df["dt"] = sensor_df["ts"].apply(to_dt)
    
    init_camera = to_dt(duration_df[duration_df["action"] == "振り上げ"].end.values[0])
    init_sensor = to_dt(init_sensor_time_str)
    
    time_delta = init_sensor - init_camera
    
    sensor_df["label"] = None

    for duration_row in duration_df.iterrows():
        if duration_row[1].action == "振り上げ":
            continue

        camera_start = to_dt(duration_row[1].start)
        camera_end = to_dt(duration_row[1].end)

        sensor_start = camera_start + time_delta
        sensor_end = camera_end + time_delta

        sensor_df.loc[(sensor_df["dt"] > sensor_start) & (sensor_df["dt"] < sensor_end), "label"] = duration_row[1].action
    sensor_df = sensor_df.drop("dt", axis=1)
    return sensor_df

In [38]:
def visualize_sensor_data(sensor_df):
    colors = {
        None: "black",
        "歩いている": "cyan",
        "立っている": "magenta",
        "階段上り" : "red",
        "走っている": "yellow",
        "階段降り"  : "green",
        "座っている" : "blue",
    }
    
    c=sensor_df['label'].map(colors)
    plt.scatter(sensor_df["ts"], sensor_df["az"], c=sensor_df['label'].map(colors))
#     plt.legend()
    
#     plt.tight_layout()

In [39]:

for sensor_path, init_sensor_time_str, save_path in zip(sensor_path_list, init_sensor_list, sensor_save_path_list):
    df = read_one_sensor_file(sensor_path, init_sensor_time_str)
    print(df["label"].value_counts())
    print()
#     visualize_sensor_data(df)
    df.to_csv(save_path, index=None, encoding="shift-jis")
    


/var/folders/m8/y6jc4w1s4cbb3830x0n6zygr0000gn/T/ipykernel_91656/2549283500.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sensor_df = pd.read_csv(sensor_path, header=None)


座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64

座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64

座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64

座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64

座っている    6872
立っている    6593
歩いている    6131
階段上り     6128
走っている    5979
階段降り     5156
Name: label, dtype: int64



## Second time

In [40]:
df = pd.read_csv(duration2_file, sep="\t", header=None)
duration_df = df.rename({
    2: "start",
    3: "end",
    4: "duration",
    5: "action"
}, axis=1).drop([0, 1], axis=1)
duration_df

,start,end,duration,action
0,00:00:25.540,00:00:25.700,00:00:00.160,振り上げ
1,00:02:53.440,00:03:54.440,00:01:01.000,階段降り
2,00:04:00.780,00:05:02.850,00:01:02.070,階段上り
3,00:05:13.770,00:06:17.660,00:01:03.890,立っている
4,00:06:22.810,00:07:24.570,00:01:01.760,走っている
5,00:07:29.750,00:08:32.090,00:01:02.340,歩いている
6,00:08:58.740,00:10:00.390,00:01:01.650,座っている


In [41]:
sensor_path_list = list(map(lambda x: os.path.join(sensor_root_dir, f"Conv-sensor{x}_2-1.csv"), "ABCDE"))
sensor_path_list

['../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorA_2-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorB_2-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorC_2-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorD_2-1.csv',
 '../../data_xaicha/時刻変換済みセンサデータ/Conv-sensorE_2-1.csv']

In [42]:
init_sensor_list = [
    "17:50:23.680", #A
    "17:50:24.450", #B
    "17:50:23.470", #C
    "17:50:23.570", #D
    "17:50:23.700", #E
]

In [43]:
save_dir = os.path.join(sensor_root_dir, "..", "tran_labeled")
os.makedirs(save_dir, exist_ok = True)

sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}_2-1.csv"), "ABCDE"))

In [45]:
for sensor_path, init_sensor_time_str, save_path in zip(sensor_path_list, init_sensor_list, sensor_save_path_list):
    df = read_one_sensor_file(sensor_path, init_sensor_time_str)
    print(df["label"].value_counts())
    print()
#     visualize_sensor_data(df)
    df.to_csv(save_path, index=None, encoding="shift-jis")

立っている    6388
歩いている    6233
階段上り     6206
走っている    6175
座っている    6164
階段降り     6099
Name: label, dtype: int64

立っている    6388
歩いている    6233
階段上り     6206
走っている    6175
座っている    6164
階段降り     6099
Name: label, dtype: int64

立っている    6388
歩いている    6233
階段上り     6206
走っている    6175
座っている    6164
階段降り     6099
Name: label, dtype: int64

立っている    6388
歩いている    6233
階段上り     6206
走っている    6175
座っている    6164
階段降り     6099
Name: label, dtype: int64

立っている    6388
歩いている    6233
階段上り     6206
走っている    6175
座っている    6164
階段降り     6099
Name: label, dtype: int64



## Unify

In [48]:
first_sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}_1-1.csv"), "ABCDE"))
second_sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}_2-1.csv"), "ABCDE"))

unify_sensor_save_path_list = list(map(lambda x: os.path.join(save_dir, f"Conv-sensor{x}-1.csv"), "ABCDE"))

In [57]:
for first_time, second_time, save_file in zip(first_sensor_save_path_list, second_sensor_save_path_list, unify_sensor_save_path_list):
    first_df = pd.read_csv(first_time, encoding="shift-jis")
    second_df = pd.read_csv(second_time, encoding="shift-jis")
    
    unify_df = pd.concat([first_df, second_df], axis=0)
    unify_df = unify_df.reset_index().drop("index", axis=1)
    
    unify_df.to_csv(save_file, index=)
    
    break
    

                  ts    ax    ay    az   gx   gy    gz label
0       17:18:30.350 -2920  9214 -2935  205  567  -641   NaN
1       17:18:30.360 -2920  9180 -2960  174  609  -665   NaN
2       17:18:30.370 -2813  9161 -2979  131  585  -714   NaN
3       17:18:30.380 -2862  9161 -3097  150  597  -769   NaN
4       17:18:30.390 -2837  9112 -3087  144  652  -757   NaN
...              ...   ...   ...   ...  ...  ...   ...   ...
209529  18:00:15.580 -2735    30  9363  168 -255  -635   NaN
209530  18:00:15.590 -2671    30  9227  -26 -280  -519   NaN
209531  18:00:15.600 -2637   -34  9349 -215 -292  -494   NaN
209532  18:00:15.610 -2559  -107  9363 -276 -438  -513   NaN
209533  18:00:15.620 -2583   -39  9193 -349 -451  -513   NaN

[209534 rows x 8 columns]


/var/folders/m8/y6jc4w1s4cbb3830x0n6zygr0000gn/T/ipykernel_91656/3260058396.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  first_df = pd.read_csv(first_time, encoding="shift-jis")
/var/folders/m8/y6jc4w1s4cbb3830x0n6zygr0000gn/T/ipykernel_91656/3260058396.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  second_df = pd.read_csv(second_time, encoding="shift-jis")
